### Getting HEFS Ensembles

In [ ]:
import httpx
import matplotlib.pyplot as plt
import pandas as pd

API_ENDPOINT = "https://api.water.noaa.gov/hefs"
LOCATION_ID = "ANAW1"
PARAMETER_ID = "QINE"
uri = API_ENDPOINT + f"/v1/hydrograph-quantiles/?location_id={LOCATION_ID}&parameter_id={PARAMETER_ID}"

In [ ]:
response = httpx.get(uri)
response.json()

In [ ]:
data_rows = []
for result in response.json()["value_set"]:
    datetime_str = result["valid_datetime"].replace("T", " ").replace("Z", "")
    parsed_datetime = pd.to_datetime(datetime_str)
    for i, value in enumerate(result["quantile_values"]):
        data_rows.append({"datetime": parsed_datetime, "quantile_index": i, "value": value})

df = pd.DataFrame(data_rows)
df_pivot = df.pivot(index="datetime", columns="quantile_index", values="value")

# Plot each quantile series
plt.figure(figsize=(10, 6))
for col in df_pivot.columns:
    plt.plot(df_pivot.index, df_pivot[col])

exceed = response.json()["metadata"]["exceedance_quantiles"]

ax = plt.subplot(111)
box = ax.get_position()
ax.set_position([box.x0, box.y0, box.width * 0.8, box.height])
plt.legend(
    exceed, title="Exceedance Probability", loc="center left", fontsize="small", bbox_to_anchor=(1, 0.5)
)

plt.xlabel("Time")
plt.ylabel("Value")
plt.title("Quantiles for Different Probabilities")
plt.xticks(rotation="vertical")
plt.show()